# 📦 GSE182159 실제 데이터 전처리

**목적**: GSE182159_RAW.tar 압축 해제 및 h5ad 변환

---

## 📋 워크플로우

| STEP | 내용 |
|------|------|
| 1 | 환경 설정 |
| 2 | tar 파일 압축 해제 |
| 3 | 샘플 구조 확인 |
| 4 | 메타데이터 로드 |
| 5a | gz 파일 해제 |
| 5b | 배치별 데이터 로드 및 저장 |
| 5c | 배치 파일 통합 |
| 5d | 메타데이터 매핑 (HBV 상태 추가) |
| 6 | h5ad로 저장 |

⚠️ **사전 조건**: `HBV_data_download.ipynb`로 GSE182159_RAW.tar 다운로드 완료

In [ ]:
# @title STEP 1: 환경 설정
import os, sys
from google.colab import drive
drive.mount('/content/drive')

RAW_PATH = '/content/drive/MyDrive/tahoe-x1/viral_hepatitis/raw_data'
EXTRACT_PATH = f'{RAW_PATH}/extracted'
os.makedirs(EXTRACT_PATH, exist_ok=True)

!pip install scanpy anndata GEOparse pandas -q

import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import gzip
import gc

print(f'\n✅ STEP 1 완료!')
print(f'📁 RAW 경로: {RAW_PATH}')
print(f'📁 추출 경로: {EXTRACT_PATH}')

In [ ]:
# @title STEP 2: tar 파일 압축 해제
import tarfile

tar_file = f'{RAW_PATH}/GSE182159_RAW.tar'

if os.path.exists(tar_file):
    # 이미 압축 해제되었는지 확인
    existing_files = os.listdir(EXTRACT_PATH) if os.path.exists(EXTRACT_PATH) else []
    gz_files = [f for f in existing_files if f.endswith('.gz') or f.endswith('.txt')]
    
    if len(gz_files) >= 40:
        print(f'⏭️ 이미 압축 해제됨 ({len(gz_files)}개 파일 존재)')
    else:
        print(f'📦 압축 해제 중: {tar_file}')
        print(f'   크기: {os.path.getsize(tar_file)/1e9:.2f} GB')
        
        with tarfile.open(tar_file, 'r') as tar:
            tar.extractall(path=EXTRACT_PATH)
        
        print('\n✅ 압축 해제 완료!')
else:
    print(f'⚠️ 파일이 없습니다: {tar_file}')
    print('   HBV_data_download.ipynb를 먼저 실행하세요.')

In [ ]:
# @title STEP 3: 샘플 구조 확인
print('📁 압축 해제된 파일 목록:')

files = os.listdir(EXTRACT_PATH)
print(f'   총 {len(files)}개 파일\n')

# 파일 유형별 분류
file_types = {}
for f in files:
    ext = f.split('.')[-1] if '.' in f else 'folder'
    file_types[ext] = file_types.get(ext, 0) + 1

print(f'📊 파일 유형:')
for ext, count in file_types.items():
    print(f'   .{ext}: {count}개')

# 샘플 예시
print(f'\n📄 샘플 파일 (처음 5개):')
for f in sorted(files)[:5]:
    print(f'   {f}')

In [ ]:
# @title STEP 4: 메타데이터 로드 (SOFT 파일 또는 CSV)
metadata_df = None

# 먼저 저장된 CSV 확인
csv_meta_file = f'{RAW_PATH}/sample_metadata.csv'
soft_file = f'{RAW_PATH}/GSE182159_family.soft.gz'

if os.path.exists(csv_meta_file):
    print('📋 저장된 메타데이터 CSV 로드 중...')
    metadata_df = pd.read_csv(csv_meta_file)
    print(f'✅ 로드 완료: {len(metadata_df)}개 샘플')
    
elif os.path.exists(soft_file):
    import GEOparse
    print('📋 SOFT 파일에서 메타데이터 파싱 중...')
    gse = GEOparse.get_GEO(filepath=soft_file, silent=True)
    
    sample_info = []
    for gsm_id, gsm in gse.gsms.items():
        info = {
            'sample_id': gsm_id,
            'title': gsm.metadata.get('title', [''])[0],
            'source': gsm.metadata.get('source_name_ch1', [''])[0],
        }
        for char in gsm.metadata.get('characteristics_ch1', []):
            if ':' in char:
                key, val = char.split(':', 1)
                info[key.strip()] = val.strip()
        sample_info.append(info)
    
    metadata_df = pd.DataFrame(sample_info)
    metadata_df.to_csv(csv_meta_file, index=False)
    print(f'✅ 메타데이터 저장: {csv_meta_file}')
else:
    print('⚠️ 메타데이터 파일이 없습니다.')

if metadata_df is not None:
    print(f'\n📊 샘플 수: {len(metadata_df)}')
    print(f'📋 컬럼: {metadata_df.columns.tolist()}')
    print(f'\n📋 처음 5개 샘플:')
    print(metadata_df.head())

In [ ]:
# @title STEP 5a: 개별 샘플 파일 처리 (gz 해제)
import shutil

gz_files = [f for f in os.listdir(EXTRACT_PATH) if f.endswith('.gz')]
txt_files = [f for f in os.listdir(EXTRACT_PATH) if f.endswith('.txt')]

if len(txt_files) >= len(gz_files) and len(txt_files) >= 40:
    print(f'⏭️ 이미 gz 해제됨 ({len(txt_files)}개 txt 파일 존재)')
else:
    print(f'📦 {len(gz_files)}개 gz 파일 해제 중...')
    
    for i, gz_file in enumerate(gz_files):
        gz_path = f'{EXTRACT_PATH}/{gz_file}'
        out_path = gz_path[:-3]  # .gz 제거
        
        if not os.path.exists(out_path):
            with gzip.open(gz_path, 'rb') as f_in:
                with open(out_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
        
        if (i+1) % 10 == 0:
            print(f'   {i+1}/{len(gz_files)} 완료')
    
    print('\n✅ gz 해제 완료!')

In [ ]:
# @title STEP 5b: 배치별 데이터 로드 및 저장 (대용량 처리)
from tqdm import tqdm

txt_files = sorted([f for f in os.listdir(EXTRACT_PATH) if f.endswith('.txt')])
print(f'📊 발견된 txt 파일: {len(txt_files)}개')

# 이미 배치 파일이 있는지 확인
existing_batches = [f for f in os.listdir(RAW_PATH) if f.startswith('GSE182159_batch_')]
if len(existing_batches) >= 5:
    print(f'⏭️ 배치 파일 이미 존재 ({len(existing_batches)}개)')
    for bf in sorted(existing_batches):
        print(f'   {bf}')
else:
    BATCH_SIZE = 10
    
    for i in range(0, len(txt_files), BATCH_SIZE):
        batch_files = txt_files[i:i+BATCH_SIZE]
        batch_num = i // BATCH_SIZE
        
        print(f'\n📦 배치 {batch_num}: {len(batch_files)}개 샘플 처리 중...')
        
        adata_list = []
        for txt_file in tqdm(batch_files):
            try:
                df = pd.read_csv(f'{EXTRACT_PATH}/{txt_file}', delim_whitespace=True, index_col=0)
                df = df.T  # 전치
                
                adata_sample = ad.AnnData(df.values.astype('float32'))
                adata_sample.obs_names = df.index
                adata_sample.var_names = df.columns
                
                sample_name = txt_file.replace('.txt', '')
                adata_sample.obs['sample'] = sample_name
                adata_sample.obs['tissue'] = 'Blood' if 'Blood' in sample_name else 'Liver'
                
                adata_list.append(adata_sample)
                del df
                gc.collect()
            except Exception as e:
                print(f'⚠️ {txt_file} 처리 실패: {e}')
        
        if adata_list:
            adata_batch = ad.concat(adata_list, join='outer')
            adata_batch.write_h5ad(f'{RAW_PATH}/GSE182159_batch_{batch_num}.h5ad')
            print(f'   ✅ batch_{batch_num}.h5ad 저장 완료 (세포 수: {adata_batch.n_obs:,})')
            
            del adata_list, adata_batch
            gc.collect()
    
    print('\n🎉 모든 배치 저장 완료!')

In [ ]:
# @title STEP 5c: 배치 파일 통합
batch_files = sorted([f for f in os.listdir(RAW_PATH) if f.startswith('GSE182159_batch_')])

if not batch_files:
    print('⚠️ 배치 파일이 없습니다. STEP 5b를 먼저 실행하세요.')
else:
    print(f'🔗 {len(batch_files)}개 배치 통합 중...')
    print('   (이 작업은 몇 분이 소요될 수 있습니다)\n')
    
    adata_batch_list = []
    for bf in batch_files:
        print(f'   로드: {bf}')
        adata_batch_list.append(ad.read_h5ad(f'{RAW_PATH}/{bf}'))
    
    adata = ad.concat(adata_batch_list, join='outer')
    adata.var_names_make_unique()
    
    print(f'\n✅ 통합 완료!')
    print(f'   세포 수: {adata.n_obs:,}')
    print(f'   유전자 수: {adata.n_vars:,}')
    print(f'   샘플 수: {adata.obs["sample"].nunique()}')
    
    del adata_batch_list
    gc.collect()

In [ ]:
# @title STEP 5d: 메타데이터 매핑 (HBV 상태 추가)

# adata 확인
if 'adata' not in dir() or adata is None:
    print('⚠️ adata가 없습니다. STEP 5c를 먼저 실행하세요.')
else:
    # metadata_df 로드 (없으면 CSV에서 로드)
    if 'metadata_df' not in dir() or metadata_df is None:
        csv_meta_file = f'{RAW_PATH}/sample_metadata.csv'
        if os.path.exists(csv_meta_file):
            metadata_df = pd.read_csv(csv_meta_file)
            print(f'📋 메타데이터 CSV 로드: {len(metadata_df)}개 샘플')
        else:
            print('⚠️ 메타데이터가 없습니다. STEP 4를 먼저 실행하세요.')
            metadata_df = None
    
    if metadata_df is not None:
        print('🔗 메타데이터 매핑 중...')
        
        # GSM ID 추출 (샘플명에서)
        adata.obs['gsm_id'] = adata.obs['sample'].str.extract(r'(GSM\d+)')[0]
        
        # 매핑용 딕셔너리 생성
        meta_indexed = metadata_df.set_index('sample_id')
        
        # 각 컬럼 매핑
        for col in metadata_df.columns:
            if col != 'sample_id':
                adata.obs[col] = adata.obs['gsm_id'].map(meta_indexed[col].to_dict())
        
        # HBV 상태 확인
        print('\n📊 추가된 메타데이터:')
        print(f'   컬럼: {list(adata.obs.columns)}')
        
        if 'Stage' in adata.obs.columns:
            print(f'\n📊 HBV Stage 분포:')
            print(adata.obs['Stage'].value_counts())
        
        if 'tissue' in adata.obs.columns:
            print(f'\n📊 Tissue 분포:')
            print(adata.obs['tissue'].value_counts())
        
        print('\n✅ 메타데이터 매핑 완료!')

In [ ]:
# @title STEP 6: h5ad로 저장
if 'adata' not in dir() or adata is None:
    print('⚠️ 저장할 데이터가 없습니다. STEP 5c를 먼저 실행하세요.')
else:
    output_file = f'{RAW_PATH}/GSE182159_processed.h5ad'
    
    print(f'💾 저장 중: {output_file}')
    print('   (이 작업은 몇 분이 소요될 수 있습니다)\n')
    
    adata.write_h5ad(output_file)
    
    print('=' * 60)
    print('🎉 전처리 완료!')
    print('=' * 60)
    print(f'\n📁 저장된 파일: {output_file}')
    print(f'\n📊 데이터 요약:')
    print(f'   세포 수: {adata.n_obs:,}')
    print(f'   유전자 수: {adata.n_vars:,}')
    print(f'   샘플 수: {adata.obs["sample"].nunique()}')
    print(f'\n📋 메타데이터 컬럼:')
    print(f'   {list(adata.obs.columns)}')
    print('\n✅ 다음 단계: Step1_HBV_real.ipynb 실행')

In [ ]:
# @title (선택) 저장된 파일 검증
output_file = f'{RAW_PATH}/GSE182159_processed.h5ad'

if os.path.exists(output_file):
    print('🔍 저장된 파일 검증 중...')
    adata_check = ad.read_h5ad(output_file, backed='r')
    print(f'   Shape: {adata_check.shape}')
    print(f'   Obs columns: {list(adata_check.obs.columns)}')
    
    if adata_check.n_obs > 0:
        print(f'\n✅ 검증 완료! 세포 수: {adata_check.n_obs:,}')
    else:
        print(f'\n❌ 경고: 세포 수가 0입니다! STEP 5c부터 다시 실행하세요.')
else:
    print(f'⚠️ 파일이 없습니다: {output_file}')